In [ ]:
!gdown 1r_KmPlYg_s2goHu1pOvSWZOKYMQxlCuP
!gdown 1osE3vJr_CHwbk8CeX3tBcLNJv4btcgaw
!gdown 1afdjwNCyk4s5FIOuiM6_3eg531J5cviX

!unzip 'imgs-for-ai.zip'
!unzip 'imgs-for-ai-part-2.zip'

In [ ]:
!pip install torcheval

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 5.1 MB/s eta 0:00:00


In [ ]:
import torch
from torcheval.metrics.functional import multiclass_f1_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torchvision.transforms as T
from torch.utils.data import random_split
from PIL import Image
from torchvision import transforms
from tqdm.auto import tqdm
import torch.optim as optim
import os
import torch
import torch.nn as nn
import torchvision.models as models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
targets = pd.read_csv('/content/ classifier_targets.csv', header=None)

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, data, fact_def_path='/', target_def_path='/', transform=None):
        self.target_paths = [data.iloc[i, 1] for i in range(len(data))]
        self.fact_paths = [data.iloc[i, 0] for i in range(len(data))]
        self.labels = [data.iloc[i, 2] for i in range(len(data))]
        self.fact_def_path = fact_def_path
        self.target_def_path = target_def_path
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        target_img = Image.open(self.target_def_path + self.target_paths[idx])
        fact_img = Image.open(self.fact_def_path + self.fact_paths[idx])

        target_img = self.transform(target_img)
        fact_img = self.transform(fact_img)

        transform = transforms.ToTensor()
        target_img = transform(target_img)
        fact_img = transform(fact_img)

        if target_img.shape[0] == 4:
            target_img = target_img[:-1]
        if fact_img.shape[0] == 4:
            fact_img = fact_img[:-1]

        y = self.labels[idx]

        return target_img, fact_img, y

In [ ]:
image_size = (256, 256)

transform = transforms.Compose([
    transforms.Resize(image_size),
])

In [ ]:
batch_size = 32

dataset = Dataset(targets, fact_def_path='/content/imgs/', target_def_path='/content/contents/', transform=transform)

train_data, val_data = random_split(dataset, [0.99, 0.01])

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=True)

In [ ]:
class DualResNet18Classifier(nn.Module):
    def __init__(self):
        super(DualResNet18Classifier, self).__init__()

        efficientnet = models.efficientnet_b0(pretrained=True)

        self.feature_extractor = nn.Sequential(*list(efficientnet.children())[:-1])

        self.fc = nn.Sequential(
            nn.Linear(2560, 256),
            nn.ReLU(),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, img1, img2):
        feat1 = self.feature_extractor(img1)
        feat2 = self.feature_extractor(img2)

        feat1 = feat1.view(feat1.size(0), -1)
        feat2 = feat2.view(feat2.size(0), -1)

        combined_features = torch.cat((feat1, feat2), dim=1)
        output = self.fc(combined_features)

        return output

model = DualResNet18Classifier()
model.to(device)
pass

In [ ]:
num_epochs = 25

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

for epoch in tqdm(range(num_epochs)):
    model.train()
    running_loss = 0.0

    for img1, img2, labels in train_loader:
        img1, img2, labels = img1.to(device), img2.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(img1, img2).squeeze()
        loss = criterion(outputs, labels.float())

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * img1.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

  0%|          | 0/25 [00:00<?, ?it/s]

Epoch [1/25], Loss: 0.6310
Epoch [2/25], Loss: 0.4022
Epoch [3/25], Loss: 0.4396
Epoch [4/25], Loss: 0.2293
Epoch [5/25], Loss: 0.2369
Epoch [6/25], Loss: 0.2381
Epoch [7/25], Loss: 0.1331
Epoch [8/25], Loss: 0.2402
Epoch [9/25], Loss: 0.2031
Epoch [10/25], Loss: 0.1544
Epoch [11/25], Loss: 0.1631
Epoch [12/25], Loss: 0.1457
Epoch [13/25], Loss: 0.1497
Epoch [14/25], Loss: 0.1450
Epoch [15/25], Loss: 0.1011
Epoch [16/25], Loss: 0.1498
Epoch [17/25], Loss: 0.1298
Epoch [18/25], Loss: 0.0933
Epoch [19/25], Loss: 0.0599
Epoch [20/25], Loss: 0.0848
Epoch [21/25], Loss: 0.0958
Epoch [22/25], Loss: 0.1191
Epoch [23/25], Loss: 0.0971
Epoch [24/25], Loss: 0.0363
Epoch [25/25], Loss: 0.0740


In [ ]:
torch.save(model.state_dict(), 'classifier_model.pt')